In [1]:
# @title
### EP3 MAC0417 / MAC5768 ##################################################
#  AO PREENCHER ESSE CABEÇALHO COM O MEU NOME E O MEU NÚMERO USP,              #
#  DECLARO QUE SOU O ÚNICO AUTOR E RESPONSÁVEL PELAS RESPOSTAS NESTA LISTA.    #
#  TODAS AS PARTES FORAM DESENVOLVIDAS E IMPLEMENTADAS POR MIM, SEGUINDO AS    #
#  INSTRUÇÕES E QUE PORTANTO, NÃO CONSTITUEM DESONESTIDADE ACADÊMICA OU PLÁGIO.#
#                                                                              #
#  DECLARO TAMBÉM QUE SOU RESPONSÁVEL POR TODAS AS CÓPIAS                      #
#  DESSE PROGRAMA E QUE EU NÃO DISTRIBUI OU FACILITEI A                        #
#  SUA DISTRIBUIÇÃO. ESTOU CIENTE QUE OS CASOS DE PLÁGIO E                     #
#  DESONESTIDADE ACADÊMICA SERÃO TRATADOS SEGUNDO OS CRITÉRIOS                 #
#  DEFINIDOS NO CÓDIGO DE ÉTICA DA USP.                                        #
#  ENTENDO QUE LISTAS SEM ASSINATURA NÃO SERÃO CORRIGIDAS E,                   #
#  AINDA ASSIM, PODERÃO SER PUNIDOS POR DESONESTIDADE ACADÊMICA.               #
#                                                                              #
#  Nome : Érico Tiago Meireles                                                 #
#  NUSP : 9312428                                                              #
#  Turma: MAC5768 Visão e Processamento de Imagens                             #
#  Prof.: Ronaldo Fumio Hashimoto                                              #
################################################################################

## EP3 - Parte 2 - Extração de características

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os

import cv2

pd.set_option('display.max_columns', None)  
pd.set_option('display.max_colwidth', None)

#### Importa os metadados

In [2]:
metadata = pd.read_csv(rf"../metadados.csv", sep=';')

#### Criação da tabela sumária geral:

In [3]:

## Lista todas as imagens dos dados do EP2

path_list = []
names = []
path_folder = []
for path, subdirs, files in os.walk("../data_EP2/"):
    for name in files:
        path_list.append((os.path.join(path, name)))
        path_folder.append(path.split(rf'/')[-1])
        names.append(name)

In [4]:
# Cria um dataframe com todas as imagens, diretórios e pastas do Ep2
df_images = pd.DataFrame({'path' : path_list,
                          'img' : names,
                          'folder' : path_folder
                          })

In [5]:
# Define uma nova coluna "classe" para agrupar todas as classes presentes em cada imagem

metadata['classe'] = metadata['classe_1']
metadata['n_objetos'] = 1

for i in range(2, 6):
    metadata['classe'] += ', ' + metadata.apply(lambda row: row['classe_' + str(i)] if pd.notnull(row['classe_' + str(i)]) else '', axis=1)
    metadata['n_objetos'] += metadata.apply(lambda row: 1 if pd.notnull(row['classe_' + str(i)]) else 0, axis=1)


# Remove ',' no final do string
metadata['classe'] = metadata['classe'].str.rstrip(', ')

metadata['classe'] = metadata['classe'].str.rstrip(', ')

In [6]:
metadata.head(2)

,img,classe_1,classe_2,classe_3,classe_4,classe_5,fundo,iluminacao,classe,n_objetos
0,001.JPEG,garfo,pilha,faca,colher,NaN,sofa,interno-claro,"garfo, pilha, faca, colher",4
1,002.JPEG,som,livro,capacete,NaN,NaN,sofa,interno-claro,"som, livro, capacete",3


In [7]:
# Separa as palavras por vírgula e cria uma lista única de palavras
palavras_unicas = set(palavra.strip() for palavra in ','.join(metadata['classe']).split(',') if palavra)
# Converte o conjunto de volta para uma lista
lista_palavras_unicas = list(palavras_unicas)

##### TABELA SUMÁRIA

In [8]:

# Define type como o tipo de transformação utilizada
df_images['type'] = df_images['path'].str.split(rf'\\').str[-2]
df_images['type'] = df_images['type'].str.split(rf'/').str[-1]
# Garantindo que o nome tenha JPEG maiúsculo
df_images['img'] = df_images['img'].str.upper()

In [9]:
# Cria a relação de todas as imagens com a sua classe e iluminação, através do metadado:

metadata = df_images.merge(metadata, on='img', how='left')

###### Criação do dataset único por imagem:

In [19]:
metadata_unique = metadata.drop_duplicates('img')

### Detecção de Arestas

#### Define funções para a suavização e detecção de arestas da imagem

In [88]:
from skimage.measure import label, regionprops

# Lista para armazenar dados para o CSV

dados_csv = []

# Função para suavizar a imagem
def apply_smoothing(img, method='gaussian'):
    if method == 'gaussian':
        return cv2.GaussianBlur(img, (5, 5), 0)
    elif method == 'median':
        return cv2.medianBlur(img, 5)
    elif method == 'bilateral':
        return cv2.bilateralFilter(img, 9, 75, 75)
    else:
        raise ValueError('Unknown smoothing method.')

# Função para detecção de bordas gaussian, median, bilateral, gradient, canny, 'marr_hildreth'
def edge_detection(img, method='canny'):
    if method == 'gradient':
        grad_x = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=3)
        grad_mag = np.sqrt(grad_x**2 + grad_y**2)
        grad_mag = np.uint8(grad_mag / np.max(grad_mag) * 255)
        _, binary_img = cv2.threshold(grad_mag, 100, 255, cv2.THRESH_BINARY)
        return binary_img
    elif method == 'canny':
        return cv2.Canny(img, 50, 150)
    elif method == 'marr_hildreth':
        blurred = cv2.GaussianBlur(img, (0, 0), 1.0)
        log = cv2.Laplacian(blurred, cv2.CV_64F)
        log = np.uint8((log - log.min()) / (log.max() - log.min()) * 255)
        _, binary_img = cv2.threshold(log, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        return binary_img
    else:
        raise ValueError('Unknown edge detection method.')

# Função para traçar os contornos, para cada contorno utiliza uma cor
def draw_contours(img, contours, i):
    img_contours = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    if i == 0:
        cv2.drawContours(img_contours, contours, -1, (255, 0, 0), 2)
    elif i == 1:
        cv2.drawContours(img_contours, contours, -1, (0, 255, 0), 2)
    elif i == 2:
        cv2.drawContours(img_contours, contours, -1, (0, 0, 255), 2)
    elif i == 3:
        cv2.drawContours(img_contours, contours, -1, (255, 255, 0), 2)
    elif i == 4:
        cv2.drawContours(img_contours, contours, -1, (255, 0, 255), 2)
    else:
        cv2.drawContours(img_contours, contours, -1, (0, 255, 255), 2)
    return img_contours

# Função para filtrar contornos próximos das bordas
def filter_contours_near_edges(contours, img_shape, margin=10):
    height, width = img_shape
    filtered_contours = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        if x > margin and y > margin and (x + w) < (width - margin) and (y + h) < (height - margin):
            filtered_contours.append(contour)
    return filtered_contours


### Define as características dos objetos localizados na imagem

##### Para cada imagem já selecionada através do código anterior, vamos aplicar as transformações de suavização e detecção de bordas
##### Além de criar um dataframe com as informações de área, diâmetro e menor axis

In [91]:

# Função principal para processar as imagens
def process_images(input_dir, output_dir, smoothing_method='gaussian', edge_method='canny'):
    
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            img_path = os.path.join(root, file)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)


            if img is not None:
                print(file.upper())

                # Aplicar a suavização
                img_smoothed = apply_smoothing(img, smoothing_method)
                n = int(metadata_unique.loc[metadata_unique['img'] == file.upper()]['n_objetos'].item()) #
                    
                    
                # Aplicar a detecção de bordas
                img_edges = edge_detection(img_smoothed, edge_method)

                # Encontrar contornos
                contours, hierarchy = cv2.findContours(img_edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

                # Filtrar contornos próximos das bordas
                contours = filter_contours_near_edges(contours, img.shape)

                label_img = label(img_smoothed, connectivity=2)
                props = regionprops(label_img)

                # Calcular a área dos contornos
                areas = []
                for c in contours:
                    areas.append(cv2.contourArea(c))


                areas_sorted = sorted(areas, reverse=True)

                major_contours = []

                # try:

                for c in contours:
                    if len(areas_sorted) >= n:
                        if (cv2.contourArea(c)) > float(areas_sorted[n]):
                            major_contours.append(c)
                    else:
                        if (cv2.contourArea(c)) > len(areas_sorted):
                            major_contours.append(c)

                for j in range(len(major_contours)):
                    print(j)
                    # Desenhar os contornos maiores na imagem original 
                    img_contours = draw_contours(img, major_contours[j], j)
                    
                    # Salvar as imagens processadas de cada contorno para auxiliar na classificação dos objetos
                    sub_dir = os.path.relpath(root, input_dir)
                    output_sub_dir = os.path.join(output_dir, sub_dir, smoothing_method + '_' + edge_method)
                    
                    if not os.path.exists(output_sub_dir):
                        os.makedirs(output_sub_dir)
                    
                    base_filename = os.path.splitext(file)[0]                 

                    cv2.imwrite(os.path.join(output_sub_dir, base_filename + '_' + str(j) +'.JPEG'), img_contours)


                # # Filtrar props com base nas áreas dos maiores contornos
                props_filtradas = [prop for prop in props if prop.area in areas]
                props_img = props_filtradas

                # Ordenar as áreas do maior para o menor valor
                object_sorted = sorted(zip(areas, props_img), key=lambda x: x[0], reverse=True)
                

                maiores_areas_props = areas_sorted[:n]
                maiores_obj_props = object_sorted[:n]


                for idx, (area, prop) in enumerate(maiores_obj_props):
                    if area is not None and prop is not None:
                        try:
                            diameter = prop.major_axis_length  
                            minor_axis = prop.minor_axis_length  
                            centroid = prop.centroid  
                            centroid_x, centroid_y = centroid[1], centroid[0]  
                        except ValueError as e:
                            print(f"Erro ao calcular propriedades: {e}")
                            diameter = None
                            minor_axis = None           
                            centroid_x = None
                            centroid_y = None
                    else:
                        area = ''
                        diameter = ''
                        minor_axis = ''
                        centroid_x = ''
                        centroid_y = ''
                
                    # Adicionar os dados para o CSV
                    dados_csv.append({
                        'img': file,  
                        'path_origin': img_path,
                        'Objeto': '',  # Será preenchido manualmente
                        'Classe': '',  # Será preenchido através dos metadados
                        'n_contorno': idx,
                        'Área': f'{area:.10f}',
                        'Diâmetro': f'{diameter:.15f}',
                        'Minor': f'{minor_axis:.15f}',
                        'Centroide_X': f'{centroid_x:.4f}',
                        'Centroide_Y': f'{centroid_y:.4f}'
                    })

    return dados_csv

## Caminho das imagens originais selecionadas
caminho_selected_imgs_original = rf'..\data_EP3\selectedData\origin'

## Caminho onde será salvo as imagens de cada contorno
caminho_selected_imgs_output = rf'..\data_EP3\selectedData\contours'

data_characteristics = process_images(caminho_selected_imgs_original,caminho_selected_imgs_output , 'gaussian', 'canny')

006.JPEG
0
1
2
3
007.JPEG
0
1
2
3
4
009.JPEG
0
1
2
014.JPEG
0
1
2
3
015.JPEG
0
1
2
3
016.JPEG
0
1
2
3
017.JPEG
0
1
2
3
019.JPEG
0
1
2
3
4
020.JPEG
0
1
2
3
021.JPEG
0
1
2
3
022.JPEG
0
1
2
3
028.JPEG
0
1
2
3
029.JPEG
0
1
2
031.JPEG
0
1
2
3
032.JPEG
0
1
2
3
041.JPEG
0
1
2
043.JPEG
0
1
2
044.JPEG
0
1
2
3
4
047.JPEG
0
1
2
3
049.JPEG
0
1
2
050.JPEG
0
1
2
3
052.JPEG
0
1
2
055.JPEG
0
1
2
3
056.JPEG
0
1
2
3
058.JPEG
0
1
2
059.JPEG
0
1
2
3
064.JPEG
0
1
2
065.JPEG
0
1
2
3
067.JPEG
0
1
2
3
068.JPEG
0
1
2
3
073.JPEG
0
1
2
3
074.JPEG
0
1
2
3
4
075.JPEG
0
1
2
3
4
076.JPEG
0
1
2
3
079.JPEG
0
1
2
3
080.JPEG
0
1
2
3
4
081.JPEG
0
1
2
3
082.JPEG
0
1
2
3
4
085.JPEG
0
1
2
3
4
092.JPEG
0
1
2
021.JPEG
0
1
2
3
052.JPEG
0
1
2
006.JPEG
0
1
2
3
007.JPEG
0
1
2
3
4
008.JPEG
0
1
2
015.JPEG
0
1
2
3
021.JPEG
0
1
2
3
022.JPEG
0
1
2
3
028.JPEG
0
1
2
3
030.JPEG
0
1
2
3
037.JPEG
0
1
2
3
038.JPEG
0
1
2
3
4
041.JPEG
0
1
2
043.JPEG
0
1
2
044.JPEG
0
1
2
3
4
047.JPEG
0
1
2
3
050.JPEG
0
1
2
3
052.JPEG
0
1
2
056.JPEG
0
1
2
3
065

In [57]:
## define o dataframe
df_characteristics = pd.DataFrame(data_characteristics)

In [59]:
df_characteristics.head(10)

,img,path_origin,Objeto,Classe,n_contorno,Área,Diâmetro,Minor,Centroide_X,Centroide_Y
0,006.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\exponencial\006.JPEG,,,0,268.0000000000,8.115760077963412,3.474469507268025,775.0000,5.4375
1,006.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\exponencial\006.JPEG,,,1,246.5000000000,49.062238337748624,6.093960964122037,54.0676,9.9730
2,006.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\exponencial\006.JPEG,,,2,77.0000000000,14.715306997540758,3.825348832085284,251.1364,7.1818
3,006.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\exponencial\006.JPEG,,,3,74.0000000000,11.038487529760703,9.731148721328724,346.1622,7.7027
4,007.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\exponencial\007.JPEG,,,0,18194.5000000000,0.000000000000000,0.000000000000000,83.0000,0.0000
5,007.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\exponencial\007.JPEG,,,1,18083.0000000000,4.828427124746190,0.828427124746190,91.5000,1.5000
6,007.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\exponencial\007.JPEG,,,2,2132.5000000000,2.000000000000000,0.000000000000000,58.5000,0.0000
7,007.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\exponencial\007.JPEG,,,3,2087.0000000000,2.828427124746190,0.000000000000000,59.5000,0.5000
8,007.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\exponencial\007.JPEG,,,4,1765.5000000000,2.000000000000000,0.000000000000000,35.0000,0.5000
9,009.JPEG,..\data_EP3\selectedData\origin\augmentedDataset\exponencial\009.JPEG,,,0,9368.0000000000,6.414370635891564,1.935132614138100,298.5000,2.3333


In [60]:
# Salva o dataframe
## Caso queira salvar, descomentar a linha abaixo
# df_characteristics.to_csv('..\image_characteristics.csv', index=False)

### Incluir as informações de Classe no dataframe de objetos/contorno 

In [73]:
metadata_unique = metadata.drop_duplicates('img')
metadata_unique.rename(columns={'classe':'Classe'}, inplace=True)
df_characteristics.drop(columns=['Classe'], inplace=True)
df_characteristics['img'] = df_characteristics['img'].str.upper()
df_charact = df_characteristics.merge(metadata_unique, on='img', how='left')

In [104]:
# Para salvar o arquivo Excel, descomente a linha abaixo

# df_charact.to_excel('..\image_characteristics.xlsx', index=False)

Manualmente os objetos do arquivo acima foram classificados com auxílio das imagens do diretório contours, gerada nesse código. Ao avaliar essas imagens com os contornos desenhados separadamente, pode-se classificar cada contorno conforme o objeto correspondente.

Importante ressaltar que manualmente foi classificado alguns contornos como "errados", geralmente backgrounds que tinham sido selecionados como contornos. Também importante comentar que NENHUMA imagem segmentada retornou todos os objetos, os contornos sempre apareciam 2 ou até 3 vezes no mesmo objeto, dessa forma, uma mesma imagem gerou diferentes linhas para o mesmo objeto.